In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
%matplotlib inline
import pandas as pd

In [2]:
import json
from pprint import pprint

def getGameJSON(path):
    with open(path) as data_file:    
        data = json.load(data_file)
    return data
        
data = getGameJSON('GSWvsCLE.json')


In [3]:
def getHomeVisitorIds(data):
    visitors = data['events'][0]['visitor']['players']
    home = data['events'][0]['home']['players']

    visitor_ids = []
    home_ids = []


    for visitor in visitors:
        visitor_ids.append(visitor['playerid'])    

    for homie in home:
        home_ids.append(homie['playerid'])  

    return {'visitor': visitor_ids, 'home': home_ids}

getHomeVisitorIds(data)

{'home': [201567,
  2592,
  202681,
  202697,
  2747,
  203521,
  203099,
  202684,
  2760,
  202389,
  2544,
  2210,
  2590],
 'visitor': [201574,
  201575,
  201578,
  2738,
  202691,
  101106,
  2571,
  203546,
  203110,
  201939,
  203105,
  2733,
  203084]}

In [4]:
# read in the court png file
court = plt.imread("fullcourt.png")

headers = ["team_id", "player_id", "x_loc", "y_loc", 
           "radius"]

def playerDFtoList(frame):
    xList = frame['x_loc'].tolist()
    yList = frame['y_loc'].tolist()
    bothList = []
    for i in range(len(xList)):
        bothList.append([ xList[i], yList[i] ])
    return np.array(bothList)

In [5]:
def getMoments(data):
    player_moments = []
    for event in data['events']:
        for moment in event['moments']:
            for player in moment[5]:
                player_moments.append(player)

    df = pd.DataFrame(player_moments, columns=headers)
    return df


In [6]:
def calculateDistance(x1,y1,x2,y2):
     dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
     return dist

def distanceAndHandler(moment):
    
    df = getMoments(moment)
    df['DistanceToBall'] = 0.0
    df['DistanceToBasket'] = 0.0
    df['HandlerId'] = 0.0
    handler = ''
    shortest_distance = 100.0
    basket1 = [5.32, 24.8]
    basket2 = [88.55, 24.8]
    halfcourt = 47.0
    
    num_plays = 4
        
    for j in range(0,num_plays):
        index = j + j*10
        
        play = df[index : index + 10]
                
        ballX = df.x_loc[index]
        ballY = df.y_loc[index]

        if ballX > halfcourt:
            dist = calculateDistance(ballX, ballY, basket2[0], basket2[1])
        elif ballX < halfcourt:
            dist = calculateDistance(ballX, ballY, basket1[0], basket1[1])
        
        df.DistanceToBasket[index] = dist
        
        for i in range(index, index+11):
            ballX = df.x_loc[index]
            ballY = df.y_loc[index]
            distToBall = float(calculateDistance(df.x_loc[i], df.y_loc[i], ballX, ballY))
            df.DistanceToBall[i] = float(distToBall)

            if distToBall > 0.0 and distToBall < shortest_distance:
                shortest_distance = distToBall
                handler = df.player_id[i]
    
            df.HandlerId[index] = handler
        
    return df


data = getGameJSON('GSWvsCLE.json')
a = distanceAndHandler(data)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

In [7]:
def teamOnOffense(moment):
    
    df = getMoments(moment)

    halfcourtX = 47.0
    if df.x_loc[0] > halfcourtX:
        return df.team_id[6]
    elif df.x_loc[0] < halfcourtX:
        return df.team_id[1]

data = getGameJSON('GSWvsCLE.json')
teamOnOffense(data)

1610612739

In [8]:
def distanceToBasket(ballX,ballY):
    basket1 = [5.32, 24.8]
    basket2 = [88.55, 24.8]
    halfcourt = 47.0
    
    if ballX > halfcourt:
        dist = calculateDistance(ballX, ballY, basket2[0], basket2[1])
        return dist
    elif ballX < halfcourt:
        dist = calculateDistance(ballX, ballY, basket1[0], basket1[1])
        return dist    
    
# data = getGameJSON('GSWvsCLE.json')
# df = getMoments(data)
# distanceToBasket(df[0:11])

In [9]:
min_5 = 25*60*6*11

def dist(data, ball):
    data["distanceToBall"] = np.sqrt((data["x_loc"] -ball["x_loc"])**2+ (data["y_loc"] -ball["y_loc"])**2)

def hasBall(data):
    data["hasBall"] = data["distanceToBall"].apply(lambda x: 1 if x < 1.8 else 0)
    
def transform(l, ball):
    ball.reset_index(drop=True, inplace =True)    
    result = []
    for player in l:
        player.reset_index(drop=True, inplace =True)
        dist(player, ball)
        hasBall(player)
        result.append(player[(player.hasBall == 1)][["x_loc", "y_loc"]])

    
    return pd.concat(result)

        

In [33]:
def removeRepeats(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [62]:
# Get moments from the zeroth event
player_moments = []
endsOfEvents = []
counter = -1
for event in data['events']:
    for moment in event['moments']:
        counter += 1
        for player in moment[5]:
            player_moments.append(player)
    endsOfEvents.append(counter)

df = pd.DataFrame(player_moments, columns=headers)
df = df.head(min_5)
endsOfEvents = removeRepeats(endsOfEvents)

In [46]:
# Get specific player's movements
ball = df[df.player_id==-1]

steph = df[df.player_id==201939]
klay = df[df.player_id==202691]
bogut = df[df.player_id==101106]
green = df[df.player_id==203110]
barnes = df[df.player_id==203084]

lebron = df[df.player_id==2544]
smith = df[df.player_id==2747]
love = df[df.player_id==201567]
irving = df[df.player_id==202681]
mozgov = df[df.player_id==202389]

# convert them to numpy arrays
ballA = playerDFtoList(ball)

stephA = playerDFtoList(steph)
klayA = playerDFtoList(klay)
bogutA = playerDFtoList(bogut)
greenA = playerDFtoList(green)
barnesA = playerDFtoList(barnes)

lebronA = playerDFtoList(lebron)
smithA = playerDFtoList(smith)
loveA = playerDFtoList(love)
irvingA = playerDFtoList(irving)
mozgovA = playerDFtoList(mozgov)

#Get ball radiuses
radii = ball['radius'].tolist()

In [75]:
# Get the indexes that overlap
skipRanges = []
# Only get the first 17 indices b/c we're only doing the first 5 minutes of the game
for eventEndIndex in endsOfEvents[:18]:
    lastLocation = ballA[eventEndIndex]
#     print '       Event end at '+str(eventEndIndex)+':',lastLocation
    for i in range(eventEndIndex+1,len(ballA[eventEndIndex+1:])):
        if (ballA[i] == lastLocation).all():
#             print 'matched location at '+str(i)+':',ballA[i]
            skipRanges.append([eventEndIndex, i])
            break
print 'Ranges to skip:',skipRanges

Ranges to skip: [[149, 299], [922, 1360], [1360, 1510], [1885, 2410], [2410, 2560], [2885, 3360], [3360, 3510], [3881, 4031]]


In [11]:
def calculateDistance(x1,y1,x2,y2):
     dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
     return dist


In [12]:
l = [steph,klay,bogut ,green ,barnes ,lebron,smith,love,irving,mozgov]

a = transform(l, ball)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
a = a.sort_index()
idx = np.unique(a.index, return_index=True)[1]
a= a.iloc[idx]
index = np.arange(0, min_5/11, 1)

a = a.reindex(index, fill_value=0)
a_pos = playerDFtoList(a)

In [14]:
def distance(ball):
    basket1 = [5.32, 24.8]
    basket2 = [88.55, 24.8]
    halfcourt = 47.0
    
    ball['distance1'] = np.sqrt((basket1[0] - ball['x_loc'])**2 + (basket1[1] - ball['y_loc'])**2)
    ball['distance2'] = np.sqrt((basket2[0] - ball['x_loc'])**2 + (basket2[1] - ball['y_loc'])**2)
    
distance(ball)
ball

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,team_id,player_id,x_loc,y_loc,radius,distance1,distance2
0,-1,-1,28.64068,45.40327,3.54135,31.118304,63.353148
1,-1,-1,28.52319,45.27989,3.68590,30.948569,63.424316
2,-1,-1,28.38205,45.16353,3.80584,30.765752,63.520513
3,-1,-1,28.21808,45.05520,3.89686,30.571150,63.641289
4,-1,-1,28.03211,44.95588,3.95463,30.366090,63.786162
5,-1,-1,27.82495,44.86658,3.97485,30.151955,63.954666
6,-1,-1,27.82198,44.85996,3.97489,30.145333,63.955410
7,-1,-1,27.81309,44.83949,3.97501,30.125077,63.957434
8,-1,-1,27.79828,44.80427,3.97526,30.090595,63.960475
9,-1,-1,27.77758,44.75342,3.97566,30.041336,63.964256


In [137]:
ex = {}
ex["ball"] = ballA.tolist()

ex["steph"] = stephA.tolist()
ex["klay"] = klayA.tolist()
ex["bogut"] = bogutA.tolist()
ex["green"] = greenA.tolist()
ex["barnes"] = barnesA.tolist()


ex["lebron"] = lebronA.tolist()
ex["smith"] = smithA.tolist()
ex["love"] = loveA.tolist()
ex["irving"] = irvingA.tolist()
ex["mozgov"] = mozgovA.tolist()

ex["pos"] = a_pos.tolist()

# print ex['steph'][148:152]
# print ex['steph'][299:301]

# Rebuild the dictionary with skip ranges
for player in ex:
    # Loop through the skipRanges, going in reverse so we don't mess up indices
    for thisRange in reversed(skipRanges):
        # Delete the indices to skip
        del ex[player][thisRange[0]:thisRange[1]]
# ex['steph'][148:152]

In [130]:
skipRanges

[[149, 299],
 [922, 1360],
 [1360, 1510],
 [1885, 2410],
 [2410, 2560],
 [2885, 3360],
 [3360, 3510],
 [3881, 4031]]

In [138]:
with open("dump.json", "w") as outfile:
    json.dump(ex, outfile)